
Ce notebook combine :
1. Le merge de test +les données meteo
2. Le nettoyage et la préparation des données
3. La création de features temporelles
4. La sauvegarde des datasets

In [2]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


## PARTIE 1 : MERGE DES DONNEES

## Chargement des données

In [3]:
df_test = pd.read_csv('../data/raw/test.csv')

print(df_test.head(3))

# Stations météo
df_stations = pd.read_csv('../data/raw/ListedesStationsMeteo.csv', sep=';')
print(df_stations.shape)
print(df_stations.columns.tolist())
print(df_stations.head(3))

     Id    week  region_code region_name
0  3235  201352           42      ALSACE
1  3236  201352           72   AQUITAINE
2  3237  201352           83    AUVERGNE
(62, 5)
['ID', 'Nom', 'Latitude', 'Longitude', 'Altitude']
     ID              Nom   Latitude  Longitude  Altitude
0  7005        ABBEVILLE  50.136000   1.834000        69
1  7015    LILLE-LESQUIN  50.570000   3.097500        47
2  7020  PTE DE LA HAGUE  49.725167  -1.939833         6


# Mapping région / Station

In [4]:
REGION_STATION_MAPPING = {
    'ALSACE': ['07190', '07280'],
    'AQUITAINE': ['07510', '07630'],
    'AUVERGNE': ['07460', '07380'],
    'BASSE-NORMANDIE': ['07027', '07139'],
    'BOURGOGNE': ['07280', '07255'],
    'BRETAGNE': ['07110', '07117', '07130'],
    'CENTRE': ['07255', '07149'],
    'CHAMPAGNE-ARDENNE': ['07072', '07168'],
    'CORSE': ['07761', '07790'],
    'FRANCHE-COMTE': ['07299', '07280'],
    'HAUTE-NORMANDIE': ['07037', '07020'],
    'ILE-DE-FRANCE': ['07150', '07149'],
    'LANGUEDOC-ROUSSILLON': ['07630', '07643'],
    'LIMOUSIN': ['07434', '07335'],
    'LORRAINE': ['07090', '07180'],
    'MIDI-PYRENEES': ['07630', '07627'],
    'NORD-PAS-DE-CALAIS': ['07005', '07015'],
    'PAYS DE LA LOIRE': ['07222', '07130'],
    'PICARDIE': ['07005', '07015'],
    'POITOU-CHARENTES': ['07335', '07255'],
    "PROVENCE-ALPES-COTE D'AZUR": ['07650', '07690'],
    'RHONE-ALPES': ['07481', '07482'],
}

station_region_map = []
for region, stations in REGION_STATION_MAPPING.items():
    for station in stations:
        station_region_map.append({
            'numer_sta': station,
            'region_name': region
        })

df_station_region = pd.DataFrame(station_region_map)
print (len(df_station_region), "associations station-région")
print(f"  Régions couvertes : {df_station_region['region_name'].nunique()}")
print(df_station_region.head(10))

45 associations station-région
  Régions couvertes : 22
  numer_sta      region_name
0     07190           ALSACE
1     07280           ALSACE
2     07510        AQUITAINE
3     07630        AQUITAINE
4     07460         AUVERGNE
5     07380         AUVERGNE
6     07027  BASSE-NORMANDIE
7     07139  BASSE-NORMANDIE
8     07280        BOURGOGNE
9     07255        BOURGOGNE


### ÉTAPE 3 : Charger les données météo (SYNOP)

In [5]:
import os
chemin_dossier = r"../data/raw/DonneesMeteorologiques"

print("Fichiers dans le dossier :")
tous_fichiers = os.listdir(chemin_dossier)
for f in tous_fichiers[:10]:
    print(f"  - {f}")

# Chercher les fichiers synop pour 2012 et 2013 pour le test
synop_files = sorted(glob.glob(os.path.join(chemin_dossier, "synop.201[23]*.csv")))
print(f"\nFichiers synop trouvés : {len(synop_files)}")

if len(synop_files) > 0:
    # Charger et concaténer
    synop_dfs = []
    for file in synop_files:
        # Extraire l'année et le mois du nom de fichier
        nom_fichier = os.path.basename(file).split('.')[1]  
        annee = nom_fichier[:4]  # Les 4 premiers caractères = année
        mois = nom_fichier[4:]   # Les 2 derniers = mois
        print(f"Chargement des données de {mois}/{annee}...")
        
        df = pd.read_csv(file, sep=';')
        synop_dfs.append(df)
    
    df_synop = pd.concat(synop_dfs, ignore_index=True)
    print(f"\nDonnées météo SYNOP : {df_synop.shape}")
    print(f"  Colonnes : {df_synop.columns.tolist()}")
else:
    print("Aucun fichier synop trouvé ")

Fichiers dans le dossier :
  - synop.200401.csv
  - synop.200402.csv
  - synop.200403.csv
  - synop.200404.csv
  - synop.200405.csv
  - synop.200406.csv
  - synop.200407.csv
  - synop.200408.csv
  - synop.200409.csv
  - synop.200410.csv

Fichiers synop trouvés : 24
Chargement des données de 01/2012...
Chargement des données de 02/2012...
Chargement des données de 03/2012...
Chargement des données de 04/2012...
Chargement des données de 05/2012...
Chargement des données de 06/2012...
Chargement des données de 07/2012...
Chargement des données de 08/2012...
Chargement des données de 09/2012...
Chargement des données de 10/2012...
Chargement des données de 11/2012...
Chargement des données de 12/2012...
Chargement des données de 01/2013...
Chargement des données de 02/2013...
Chargement des données de 03/2013...
Chargement des données de 04/2013...
Chargement des données de 05/2013...
Chargement des données de 06/2013...
Chargement des données de 07/2013...
Chargement des données de 08/20

In [6]:
print(len(synop_files))
print(synop_files[:5])

24
['../data/raw/DonneesMeteorologiques\\synop.201201.csv', '../data/raw/DonneesMeteorologiques\\synop.201202.csv', '../data/raw/DonneesMeteorologiques\\synop.201203.csv', '../data/raw/DonneesMeteorologiques\\synop.201204.csv', '../data/raw/DonneesMeteorologiques\\synop.201205.csv']


In [7]:
stations_of_interest = df_station_region['numer_sta'].unique().tolist()
stations_int = [int(s) for s in stations_of_interest]

synop_data_list = []
for file in synop_files:
    df_synop = pd.read_csv(file, sep=';', low_memory=False)
    df_synop_filtered = df_synop[df_synop['numer_sta'].isin(stations_of_interest)]
    
    if len(df_synop_filtered) == 0:
        df_synop_filtered = df_synop[df_synop['numer_sta'].isin(stations_int)]
    
    if len(df_synop_filtered) > 0:
        synop_data_list.append(df_synop_filtered)

df_synop_all = pd.concat(synop_data_list, ignore_index=True)
df_synop_all['date'] = pd.to_datetime(df_synop_all['date'], format='%Y%m%d%H%M%S', errors='coerce')

print(f"Données synop : {df_synop_all.shape}")
print(f"Stations uniques : {df_synop_all['numer_sta'].nunique()}")
print(f"Période : {df_synop_all['date'].min()} -> {df_synop_all['date'].max()}")

Données synop : (167760, 60)
Stations uniques : 29
Période : 2012-01-01 00:00:00 -> 2013-12-31 21:00:00


### ÉTAPE 4 : Gestion des dates (les convertir en week)

In [8]:
df_synop_all['date'] = pd.to_datetime(df_synop_all['date'], format='%Y%m%d%H%M%S', errors='coerce')

# Supprimer les lignes où la date n'a pas pu être convertie
df_synop_all = df_synop_all.dropna(subset=['date'])

df_synop_all['year'] = df_synop_all['date'].dt.isocalendar().year
df_synop_all['week'] = df_synop_all['date'].dt.isocalendar().week
df_synop_all['week_year'] = df_synop_all['year'].astype(str) + df_synop_all['week'].astype(str).str.zfill(2)
df_synop_all['week_year'] = df_synop_all['week_year'].astype(int)

print(df_synop_all['week_year'].head().tolist())

[201152, 201152, 201152, 201152, 201152]


### ÉTAPE 5 : Agrégation par région et semaine

In [9]:


# S'assurer que les IDs sont au même format
df_synop_all['numer_sta'] = df_synop_all['numer_sta'].astype(str).str.zfill(5)
df_station_region['numer_sta'] = df_station_region['numer_sta'].astype(str).str.zfill(5)

# Merger avec le mapping station->région
df_synop_all = df_synop_all.merge(df_station_region, on='numer_sta', how='inner')


# Variables météo à agréger
meteo_vars = [
    'tend', 'dd', 'ff', 't', 'td', 'u', 'vv', 'n', 'nbas', 'hbas',
    'pres', 'niv_bar', 'geop', 'tend24', 'tn12', 'tn24', 'tx12', 'tx24',
    'tminsol', 'tw', 'raf10', 'rafper', 'per', 'ht_neige', 'ssfrai',
    'perssfrai', 'rr1', 'rr3', 'rr6', 'rr12', 'rr24'
]

meteo_vars_available = [v for v in meteo_vars if v in df_synop_all.columns]
print(f"  Variables météo disponibles : {len(meteo_vars_available)}/{len(meteo_vars)}")

# Convertir en numérique
for var in meteo_vars_available:
    df_synop_all[var] = pd.to_numeric(df_synop_all[var], errors='coerce')

# Agréger par région et semaine (moyenne)
agg_dict = {var: 'mean' for var in meteo_vars_available}
df_meteo_agg = df_synop_all.groupby(['region_name', 'week_year'], as_index=False).agg(agg_dict)



  Variables météo disponibles : 31/31


### ÉTAPE 6 : Merge final avec test.csv

In [11]:
# Normaliser les noms de régions
df_test['region_name_clean'] = df_test['region_name'].str.upper().str.strip()
df_meteo_agg['region_name_clean'] = df_meteo_agg['region_name'].str.upper().str.strip()
# Merger
df_final = df_test.merge(
    df_meteo_agg,
    left_on=['region_name_clean', 'week'],
    right_on=['region_name_clean', 'week_year'],
    how='left'
)

print(f" Merge effectué : {df_final.shape}")

# Nettoyer les colonnes dupliquées
cols_to_drop = ['region_name_clean', 'week_year', 'region_name_y']
df_final = df_final.drop(columns=[c for c in cols_to_drop if c in df_final.columns])

if 'region_name_x' in df_final.columns:
    df_final = df_final.rename(columns={'region_name_x': 'region_name'})

# Sauvegarder
df_final.to_csv('../data/processed/test_synop_merged_inner.csv', index=False)

print(f"\nFichier généré : test_synop_merged_inner.csv")
print(f"  Dimensions : {df_final.shape}")
print(f"  Observations : {len(df_final)}")
print(f"  Variables météo ajoutées : {len(meteo_vars_available)}")

 Merge effectué : (2288, 38)

Fichier généré : test_synop_merged_inner.csv
  Dimensions : (2288, 35)
  Observations : 2288
  Variables météo ajoutées : 31


### Analyse du merge

In [12]:

print(f"  test.csv original : {len(df_test)} lignes")
print(f"  Après merge : {len(df_final)} lignes")
print(f"  Taux de couverture : {len(df_final)/len(df_test)*100:.2f}%")

print(f"\nRégions couvertes :")
print(df_final['region_name'].value_counts())

print(f"\nPériode couverte :")
print(f"  Semaines : {df_final['week'].min()} -> {df_final['week'].max()}")

print(f"\nValeurs manquantes :")
missing = df_final.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if len(missing) > 0:
    print(missing.head(10))
else:
    print("  Aucune valeur manquante!")

  test.csv original : 2288 lignes
  Après merge : 2288 lignes
  Taux de couverture : 100.00%

Régions couvertes :
region_name
ALSACE                        104
AQUITAINE                     104
AUVERGNE                      104
BASSE-NORMANDIE               104
BOURGOGNE                     104
BRETAGNE                      104
CENTRE                        104
CHAMPAGNE-ARDENNE             104
CORSE                         104
FRANCHE-COMTE                 104
HAUTE-NORMANDIE               104
LANGUEDOC-ROUSSILLON          104
LIMOUSIN                      104
LORRAINE                      104
MIDI-PYRENEES                 104
NORD-PAS-DE-CALAIS            104
PAYS-DE-LA-LOIRE              104
PICARDIE                      104
POITOU-CHARENTES              104
PROVENCE-ALPES-COTE-D-AZUR    104
ILE-DE-FRANCE                 104
RHONE-ALPES                   104
Name: count, dtype: int64

Période couverte :
  Semaines : 201201 -> 201352

Valeurs manquantes :
niv_bar      2288
tend24    

In [13]:
df=df_test.copy()

In [14]:
missing = pd.DataFrame({
    'missing_count': df.isnull().sum(),
    'missing_pct': df.isnull().mean() * 100
}).query("missing_count > 0").sort_values('missing_pct', ascending=False)

missing

,missing_count,missing_pct


## PARTIE 2 : NETTOYAGE ET PRÉPARATION DES DONNÉES

### Charger le fichier mergé

In [15]:
df = pd.read_csv('test_synop_merged_inner.csv')
print(df.shape)
print(df.columns.tolist())

print(df.info())

print(df.shape)
print(df.columns.tolist())


(2288, 35)
['Id', 'week', 'region_code', 'region_name', 'tend', 'dd', 'ff', 't', 'td', 'u', 'vv', 'n', 'nbas', 'hbas', 'pres', 'niv_bar', 'geop', 'tend24', 'tn12', 'tn24', 'tx12', 'tx24', 'tminsol', 'tw', 'raf10', 'rafper', 'per', 'ht_neige', 'ssfrai', 'perssfrai', 'rr1', 'rr3', 'rr6', 'rr12', 'rr24']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2288 entries, 0 to 2287
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           2288 non-null   int64  
 1   week         2288 non-null   int64  
 2   region_code  2288 non-null   int64  
 3   region_name  2288 non-null   object 
 4   tend         1976 non-null   float64
 5   dd           1976 non-null   float64
 6   ff           1976 non-null   float64
 7   t            1976 non-null   float64
 8   td           1976 non-null   float64
 9   u            1976 non-null   float64
 10  vv           1976 non-null   float64
 11  n            1976 non-null   float64
 1

### ÉTAPE 1 : Supprimer les colonnes redondantes

In [16]:
# Chercher les fichiers synop
synop_files = sorted(glob.glob(os.path.join(chemin_dossier, "synop.*.csv")))

print(len(synop_files))


154


### ÉTAPE 2 : Analyser et supprimer les colonnes avec >30% de NaN

In [17]:
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percent': (df.isnull().sum() / len(df) * 100).round(2)
})

missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Percent', ascending=False)

print(f"\nColonnes avec valeurs manquantes :")
print(missing_data)

cols_to_drop = missing_data[missing_data['Missing_Percent'] > 30]['Column'].tolist()
print(f"\nColonnes à supprimer (>30% NaN) : {len(cols_to_drop)}")
print(f"  {cols_to_drop}")

df_clean = df.drop(columns=cols_to_drop)
print(f"\n  Avant : {df.shape[1]} colonnes")
print(f"  Après : {df_clean.shape[1]} colonnes")
print(f"\nColonnes restantes ({df_clean.shape[1]}) :")
print(f"  {df_clean.columns.tolist()}")


Colonnes avec valeurs manquantes :
              Column  Missing_Count  Missing_Percent
niv_bar      niv_bar           2288           100.00
tend24        tend24           2288           100.00
geop            geop           2288           100.00
tx24            tx24           2288           100.00
tw                tw           2288           100.00
tn24            tn24           2288           100.00
raf10          raf10           1452            63.46
ssfrai        ssfrai            417            18.23
perssfrai  perssfrai            417            18.23
nbas            nbas            413            18.05
u                  u            312            13.64
vv                vv            312            13.64
tend            tend            312            13.64
dd                dd            312            13.64
ff                ff            312            13.64
tn12            tn12            312            13.64
n                  n            312            13.64
hbas      

### ÉTAPE 3 : Conversion des types et création de colonnes temporelles

In [18]:
df_clean.head()

,Id,week,region_code,region_name,tend,dd,ff,t,td,u,...,rafper,per,ht_neige,ssfrai,perssfrai,rr1,rr3,rr6,rr12,rr24
0,3235,201352,42,ALSACE,8.035714,186.250000,4.346429,280.184821,277.112500,81.901786,...,8.157143,-10.0,0.0,NaN,NaN,0.245536,0.664286,1.125000,2.646429,5.257143
1,3236,201352,72,AQUITAINE,3.303571,201.607143,4.844643,282.049107,278.773214,80.696429,...,8.841071,-10.0,0.0,0.0,-47.195122,0.219643,0.596429,1.171429,2.542857,4.564286
2,3237,201352,83,AUVERGNE,2.857143,202.321429,5.732143,281.217857,275.412500,67.500000,...,10.528571,-10.0,0.0,0.0,-45.555556,0.021429,0.078571,0.114286,0.371429,0.657143
3,3238,201352,25,BASSE-NORMANDIE,7.117117,215.765766,5.974775,280.480631,278.551802,87.882883,...,10.328829,-10.0,0.0,0.0,-45.000000,0.368182,1.014679,2.385185,4.007407,8.553846
4,3239,201352,26,BOURGOGNE,5.714286,198.839286,5.208929,280.675893,277.626786,81.839286,...,9.166964,-10.0,0.0,0.0,-45.652174,0.180357,0.545536,0.864286,2.242857,4.500000


In [19]:
def week_to_datetime(week_str):
    """Convertit 'AAAASS' en première date de la semaine"""
    year = int(str(week_str)[:4])
    week = int(str(week_str)[4:6])
    jan4 = datetime(year, 1, 4)
    week_one_monday = jan4 - pd.Timedelta(days=jan4.weekday())
    week_start = week_one_monday + pd.Timedelta(weeks=week-1)
    return week_start

df_clean['date'] = df_clean['week'].apply(week_to_datetime)
df_clean['date'] = pd.to_datetime(df_clean['date'])

print(f" Colonne 'date' créée")
print(f"  Période : {df_clean['date'].min()} → {df_clean['date'].max()}")
print(f"  Total : {(df_clean['date'].max() - df_clean['date'].min()).days} jours")

print(f"\n Types de colonnes :")
print(df_clean[['date', 'week', 'region_code']].dtypes)

 Colonne 'date' créée
  Période : 2012-01-02 00:00:00 → 2013-12-23 00:00:00
  Total : 721 jours

 Types de colonnes :
date           datetime64[ns]
week                    int64
region_code             int64
dtype: object


### ÉTAPE 4 : Imputation des valeurs manquantes par la médiane

In [20]:
remaining_missing = df_clean.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0].sort_values(ascending=False)
print("Valeurs manquantes avant imputation :")
print(remaining_missing)

print(f"\nImputation par médiane (colonnes numériques)...")
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()

for col in numeric_cols:
    if df_clean[col].isnull().sum() > 0:
        median_val = df_clean[col].median()
        df_clean[col].fillna(median_val, inplace=True)
        print(f"   {col} : {median_val:.2f}")

print(f"\nAucune valeur manquante restante : {df_clean.isnull().sum().sum() == 0}")
print(f"\nAperçu final du dataset :")
print(df_clean.describe())

Valeurs manquantes avant imputation :
ssfrai       417
perssfrai    417
nbas         413
tend         312
td           312
dd           312
ff           312
t            312
n            312
vv           312
u            312
hbas         312
tx12         312
tminsol      312
pres         312
tn12         312
per          312
rafper       312
ht_neige     312
rr1          312
rr3          312
rr6          312
rr12         312
rr24         312
dtype: int64

Imputation par médiane (colonnes numériques)...
   tend : 0.85
   dd : 193.21
   ff : 3.34
   t : 284.86
   td : 280.46
   u : 76.88
   vv : 22941.96
   n : 77.08
   nbas : 4.47
   hbas : 1404.95
   pres : 100070.44
   tn12 : 281.34
   tx12 : 288.05
   tminsol : 279.29
   rafper : 6.16
   per : -10.00
   ht_neige : 0.00
   ssfrai : 0.00
   perssfrai : -45.00
   rr1 : 0.06
   rr3 : 0.19
   rr6 : 0.33
   rr12 : 0.76
   rr24 : 1.52

Aucune valeur manquante restante : True

Aperçu final du dataset :
               Id           week  regio

### ÉTAPE 5 : Vérifier la fréquence des données

In [21]:

print(f"   Du : {df_clean['date'].min()}")
print(f"   Au : {df_clean['date'].max()}")
print(f"   Total : {(df_clean['date'].max() - df_clean['date'].min()).days} jours")

print("\nDistribution par région :")
region_counts = df_clean['region_code'].value_counts().sort_index()
print(region_counts.describe())

print(f"\nVérification des doublons (region_code + week) :")
duplicates = df_clean.duplicated(subset=['region_code', 'week'], keep=False)
print(f"   Doublons trouvés : {duplicates.sum()}")

print(f"\nSemaines disponibles : {sorted(df_clean['week'].unique())[:10]}... ({len(df_clean['week'].unique())} semaines)")

   Du : 2012-01-02 00:00:00
   Au : 2013-12-23 00:00:00
   Total : 721 jours

Distribution par région :
count     22.0
mean     104.0
std        0.0
min      104.0
25%      104.0
50%      104.0
75%      104.0
max      104.0
Name: count, dtype: float64

Vérification des doublons (region_code + week) :
   Doublons trouvés : 0

Semaines disponibles : [np.int64(201201), np.int64(201202), np.int64(201203), np.int64(201204), np.int64(201205), np.int64(201206), np.int64(201207), np.int64(201208), np.int64(201209), np.int64(201210)]... (104 semaines)


### ÉTAPE 6 : Créer la colonne saison
Créer une variable saison (Hiver/Printemps/Été/Automne) pour capturer la saisonnalité de la grippe

In [22]:


def get_season(date):
    """Détermine la saison en fonction du mois"""
    month = date.month
    if month in [12, 1, 2]:
        return 'Hiver'
    elif month in [3, 4, 5]:
        return 'Printemps'
    elif month in [6, 7, 8]:
        return 'Ete'
    else:
        return 'Automne'

df_clean['saison'] = df_clean['date'].apply(get_season)

print("Répartition par saison :")
print(df_clean['saison'].value_counts())

print(df_clean[['date', 'saison']].head(10))

Répartition par saison :
saison
Hiver        572
Automne      572
Ete          572
Printemps    572
Name: count, dtype: int64
        date saison
0 2013-12-23  Hiver
1 2013-12-23  Hiver
2 2013-12-23  Hiver
3 2013-12-23  Hiver
4 2013-12-23  Hiver
5 2013-12-23  Hiver
6 2013-12-23  Hiver
7 2013-12-23  Hiver
8 2013-12-23  Hiver
9 2013-12-23  Hiver


### ÉTAPE 7 : Créer le dataset avec variables sélectionnées

In [23]:
# variables sélectionnées basées sur l'analyse de corrélation
variables_selectionnees = ['tminsol', 'nbas', 'n', 'td', 'ff', 't', 'u', 'vv']

print(f"\nVariables à conserver (groupe réduit) :")
print(f"  - tminsol : température min du sol")
print(f"  - nbas : nébulosité des nuages niveau inférieur")
print(f"  - n : nébulosité totale")
print(f"  - td : point de rosée")
print(f"  - ff : vitesse du vent")
print(f"  - t : température")
print(f"  - u : humidité")
print(f"  - vv : visibilité horizontale")

# vérifier que toutes les variables existent
variables_disponibles = [var for var in variables_selectionnees if var in df_clean.columns]
variables_manquantes = [var for var in variables_selectionnees if var not in df_clean.columns]

print(f"\nVariables trouvées : {len(variables_disponibles)}/{len(variables_selectionnees)}")
if variables_manquantes:
    print(f"Variables non trouvées : {variables_manquantes}")

# créer df_reduit avec uniquement les variables sélectionnées
colonnes_obligatoires = [ 'region_code', 'region_name', 'week', 'date', 'saison']
df_reduit = df_clean[colonnes_obligatoires + variables_disponibles].copy()

print(f"\nDataset complet : {df_clean.shape}")
print(f"Dataset réduit (variables sélectionnées) : {df_reduit.shape}")
print(f"Colonnes du dataset réduit : {df_reduit.columns.tolist()}")


Variables à conserver (groupe réduit) :
  - tminsol : température min du sol
  - nbas : nébulosité des nuages niveau inférieur
  - n : nébulosité totale
  - td : point de rosée
  - ff : vitesse du vent
  - t : température
  - u : humidité
  - vv : visibilité horizontale

Variables trouvées : 8/8

Dataset complet : (2288, 30)
Dataset réduit (variables sélectionnées) : (2288, 13)
Colonnes du dataset réduit : ['region_code', 'region_name', 'week', 'date', 'saison', 'tminsol', 'nbas', 'n', 'td', 'ff', 't', 'u', 'vv']


### ÉTAPE 8 : Créer des features temporelles (lags et moyennes mobiles)

## SAUVEGARDE DES DATASETS FINAUX

In [24]:
#  Dataset complet (toutes les variables)
df_clean.to_csv('../data/processed/test_meteo_full.csv', index=False)
print("\n1. Dataset complet (toutes variables) :")
print(f"   Fichier : test_meteo_full.csv")
print(f"   Shape : {df_clean.shape}")
print(f"   Colonnes : {len(df_clean.columns)}")

#  Dataset réduit (variables sélectionnées)
df_reduit.to_csv('../data/processed/test_meteo_reduit.csv', index=False)
print("\n2. Dataset réduit (variables sélectionnées) :")
print(f"   Fichier : test_meteo_reduit.csv")
print(f"   Shape : {df_reduit.shape}")
print(f"   Colonnes : {df_reduit.columns.tolist()}")



1. Dataset complet (toutes variables) :
   Fichier : test_meteo_full.csv
   Shape : (2288, 30)
   Colonnes : 30

2. Dataset réduit (variables sélectionnées) :
   Fichier : test_meteo_reduit.csv
   Shape : (2288, 13)
   Colonnes : ['region_code', 'region_name', 'week', 'date', 'saison', 'tminsol', 'nbas', 'n', 'td', 'ff', 't', 'u', 'vv']


## RÉSUMÉ FINAL

In [25]:

print(f"  test_meteo_full.csv : {df_clean.shape[0]} lignes x {df_clean.shape[1]} colonnes")
print(f"  test_meteo_reduit.csv : {df_reduit.shape[0]} lignes x {df_reduit.shape[1]} colonnes")

print(f"\nVariables sélectionnées ({len(variables_disponibles)}) :")
for var in variables_disponibles:
    print(f"  - {var}")

  test_meteo_full.csv : 2288 lignes x 30 colonnes
  test_meteo_reduit.csv : 2288 lignes x 13 colonnes

Variables sélectionnées (8) :
  - tminsol
  - nbas
  - n
  - td
  - ff
  - t
  - u
  - vv
